In [134]:
import pandas as pd
xls=pd.ExcelFile('Paavan Dairy-automated-invoice.xlsx')
client_details=pd.read_excel(xls,'Clients')
product_details=pd.read_excel(xls,'Products')
Bill=pd.read_excel(xls,'Sheet1')

In [135]:
final_bill={}
flag=0
for index,row in Bill.iterrows():
    client_id=row['Client Number']
    if client_id=='nan':
        break
    product_id=row['Product']
    quantity=row['Quantity']
    name=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Name'])[1:]).strip()
    name=(name.split('\n'))[0]
    street=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Street'])[1:]).strip()
    street=(street.split('\n'))[0]
    city=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'City'])[1:]).strip()
    city=(city.split('\n'))[0]
    Phone_Number=(str(client_details.loc[client_details['Client nr']==row['Client Number'],'Phone Num'])[1:]).strip()
    Phone_Number=(Phone_Number.split('\n'))[0]
    Product=(str(product_details.loc[product_details['Product nr']==row['Product'],'Description'])[1:]).strip()
    Product=(Product.split('\n'))[0]
    Price=(str(product_details.loc[product_details['Product nr']==row['Product'],'Price'])[1:]).strip()
    Price=(Price.split('\n'))[0]
    if row['Client Number'] not in final_bill.keys():
        prod={}
        prod[0]=({'product_id':product_id,'Product':Product,'Price':Price,'Quantity':quantity})
        final_bill[row['Client Number']]={'Client_Number':row['Client Number'],'Client_Name':name,'Street':street,'City':city,'Phone_Number':Phone_Number,'prod':prod}
    else:
        prod={}
        prod=final_bill[row['Client Number']]['prod']
        product_number=prod.keys()
        count=len(product_number)
        for j in range(count):
            if prod[j]['product_id']==product_id:
                final_bill[row['Client Number']]['prod'][j]['Quantity']+=quantity
                flag=1
                break
        #print(name+str("")+str(flag))
        if flag==1:
            flag=0
        else:
            prod[count]=({'product_id':product_id,'Product':Product,'Price':Price,'Quantity':quantity})
            final_bill[row['Client Number']]={'Client_Number':row['Client Number'],'Client_Name':name,'Street':street,'City':city,'Phone_Number':Phone_Number,'prod':prod}
        

In [136]:
df=(pd.DataFrame(final_bill))

In [137]:
df=df.transpose()

In [138]:
for index,j in df.iterrows():
    j['prod']=(pd.DataFrame(j['prod']))

In [139]:
df

,City,Client_Name,Client_Number,Phone_Number,Street,prod
501,Asansol,Pratik Bansal,501,9647616532,"Bansal Bhawan, G.T. Road, Kumarpur",0 1 2 Price ...
502,Asansol,Simarjit Gandhi,502,9570109807,"Lucas Estate, Durgamandir",0 1 Price 20 ...
503,Asansol,G B Gupta,503,9999999999,Sreepally,0 Price 38 Product ...


In [140]:
import json
import random
import datetime
import pandas as pd
from pprint import pprint
file=open(r'rajat_bill.html','r')
content=file.read()

In [141]:
content=content.replace('\t',"")
content = content.replace("\n","")
    

In [142]:
for val,row in df.iterrows():
    client_id=row['Client_Number']
    cid=str(row['Client_Number'])
    city=str(row['City'])
    name=str(row['Client_Name'])
    Phone=str(row['Phone_Number'])
    street=str(row['Street'])
    prototype=""
    to ="to :"
    findto=content.find(to)
    prototype+=content[0:findto+len(to)]+" "+"<br><b>"+name+"<br>"+street+"<br>"+city+"<br>"+Phone+"</b><br>"
    date = "Date :"
    finddate=content.find(date)
    now = datetime.datetime.now()
    prototype+=content[findto+len(to):finddate+len(date)]+" "+str(now.date())

    month ="Month :"
    findmonth=content.find(month)
    prototype+=content[finddate+len(date):findmonth+len(month)]+" "+str(now.strftime("%B"))

    invoicenr = "Invoice nr :"
    findinvoicenr = content.find(invoicenr)
    prototype+=content[findmonth+len(month):findinvoicenr+len(invoicenr)]+" "+str(random.randint(1,100))

    clientnr ="Client nr :"
    findclientnr=content.find(clientnr)
    prototype+=content[findinvoicenr+len(invoicenr):findclientnr+len(clientnr)]+" "+cid

    tbody="tbody"
    findtbody=content.find(tbody)
    prototype+=content[findclientnr+len(clientnr):findtbody+len(tbody)]

    total_amount = 0
    for index,i in (row['prod'].transpose()).iterrows():
        myprice=str(i['Price'])
        myproduct=str(i['Product'])
        mypid=str(i['product_id'])
        myquantity=str(i['Quantity'])
        total = int(myquantity)*float(myprice)
        total_amount = total_amount+int(total)
        prototype+="<tr><td>"+mypid+"</td>"+"<td>"+myproduct+"</td>"+"<td>"+myprice+"</td>"+"<td>"+myquantity+"</td>"+"<td>"+str(total)+"</td></tr>"

    prototype+="</tbody></table></div>"
    prototype+='<br><br><h3 class="float-right">Total Amount Due : &#x20B9; '+str(total_amount)+"</div></body></html>"
    filename=name+".html"
    file1=open(filename,'w')
    file1.write(prototype)
    file1.close()
